In [1]:
import transformers
transformers.logging.set_verbosity(transformers.logging.CRITICAL)

import datasets
datasets.logging.set_verbosity(datasets.logging.ERROR)

import warnings
warnings.filterwarnings(category=FutureWarning ,action='ignore')

d:\Users\cdiet\miniconda3\envs\hf\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


#### Loading and Tokenizing

In [2]:
# Load and tokenized the data
from datasets import load_dataset
from transformers import AutoTokenizer, DataCollatorWithPadding

checkpoint = "bert-base-uncased"

raw_dataset = load_dataset("glue", "mrpc")

tokenizer = AutoTokenizer.from_pretrained(checkpoint)

def tokenize_function(examples):
    return tokenizer(examples["sentence1"], examples["sentence2"], truncation=True)

tokenized_datasets = raw_dataset.map(tokenize_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# Postprocess the tokenized data to make it pytorch-friendly
# - Remove columns that the model does not expect
# - Rename the column 'label' to 'labels'
# - Set the format of the dataset so they return pytorch tensors instead of lists
tokenized_datasets = tokenized_datasets.remove_columns(["sentence1", "sentence2", "idx"])
tokenized_datasets = tokenized_datasets.rename_column("label", "labels")
tokenized_datasets.set_format("torch")

100%|██████████| 3/3 [00:00<00:00, 333.53it/s]


#### Instanciate dataloaders

Previously nicely handled by the Trainer

In [3]:
from torch.utils.data import DataLoader

from torch.utils.data import DataLoader

train_dataloader = DataLoader(
    tokenized_datasets["train"], shuffle=True, batch_size=8, collate_fn=data_collator
)
eval_dataloader = DataLoader(
    tokenized_datasets["validation"], batch_size=8, collate_fn=data_collator
)

# Check one call to the train_dataloader iterator
for batch in train_dataloader:
    break
{k: v.shape for k, v in batch.items()}

{'labels': torch.Size([8]),
 'input_ids': torch.Size([8, 87]),
 'token_type_ids': torch.Size([8, 87]),
 'attention_mask': torch.Size([8, 87])}

#### Classic Training loop adapted to work with Accelerate  
- lines of code to add end with # accelerate    
- lines of code to remove are commented  

examples here: https://github.com/huggingface/accelerate/tree/main/examples  

In [ ]:
from accelerate import Accelerator # accelerate
from transformers import AdamW, AutoModelForSequenceClassification, get_scheduler
import evaluate

# Setup the compute device
# device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
# model.to(device)

# Instanciate the accelerator
accelerator = Accelerator() # accelerate

# Instanciate the model
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

# Instanciate the optimizer
optimizer = AdamW(model.parameters(), lr=3e-5)

# Prepare data for acceleration
train_dataloader, eval_dataloader, model, optimizer = accelerator.prepare(train_dataloader, eval_dataloader, model, optimizer) # accelerate

# Instanciate the scheduler
num_epochs = 3
num_training_steps = num_epochs * len(train_dataloader)
lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps,
)

# Get the evaluation metric
metric = evaluate.load("glue", "mrpc")

# Get the progress bar
progress_bar = tqdm(range(num_training_steps))

# training loop
model.train()
for epoch in range(num_epochs):
    for batch in train_dataloader:
        # batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        # loss.backward()
        accelerator.backward(loss)

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

# Evaluation loop
model.eval()
for batch in eval_dataloader:
    batch = {k: v.to(device) for k, v in batch.items()}
    with torch.no_grad():
        outputs = model(**batch)

    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)
    metric.add_batch(predictions=predictions, references=batch["labels"])
metric.compute()

In [ ]:
# Launch distributed training
# accelerate config
# accelerate launch train.py